In [1]:
import MeCab
import itertools

In [2]:
class wordvec_lib:
    
    def _word_attrs(self, txt):
        m = MeCab.Tagger("mecab-ipadic-neologd")
        line = m.parse(txt).split('\n')[:-2]
        return [l.split('\t')[1].split(',') for l in line]

    def _filter_words(self, word_attrs):
        # filter noun and verb
        w = word_attrs
        w = [m for m in w if m[0] in ['名詞', '動詞']]
        return w

    def _attr_to_word_vec(self, attrs):
        count = len(attrs)
        words = [a[6] for a in attrs]
        return {g[0] : len(list(g[1]))/count for g in itertools.groupby(sorted(words))}

    def get_word_vec(self, txt):
        attrs = self._word_attrs(txt)
        attrs = self._filter_words(attrs)
        return self._attr_to_word_vec(attrs)
    
    def dot_product(self, v1, v2):
        return sum(v1[key]*v2.get(key, 0) for key in v1)

In [3]:
t1 = "照明はボタン１で、エアコンはボタン２で操作します"
t2 = "照明はボタン１で操作します"
t3 = "エアコンはオンになっています"
w = wordvec_lib()
v1 = w.get_word_vec(t1)
v2 = w.get_word_vec(t2)
v3 = w.get_word_vec(t3)
print(w.dot_product(v1, v2))
print(w.dot_product(v1, v3))

0.15
0.03125


In [12]:
qa_list = [('照明のつけ方は？', '照明をつけるにはボタン１を押します'),
           ('エアコンのつけ方は？', 'エアコンをつけるにはボタン２を押します'),
           ('エアコンの温度の上げ方は？', 'エアコンの温度を上げるにはボタン４を押します'),
           ('テレビのつけ方は？', 'テレビをつけるにはボタン３を押します')]
q = 'エアコンの温度設定はどうやるの？'
w = wordvec_lib()
word_vec_db = [w.get_word_vec(q[0]) for q in qa_list]
word_vec_q = w.get_word_vec(q)
inner_prods = [w.dot_product(v, word_vec_q) for v in word_vec_db]
print(inner_prods)
print(word_vec_q)
print("--")
word_vec_db

[0.0, 0.08333333333333333, 0.125, 0.0]
{'温度': 0.25, 'エアコン': 0.25, '設定': 0.25, 'やる': 0.25}
--


[{'つけ': 0.3333333333333333, '方': 0.3333333333333333, '照明': 0.3333333333333333},
 {'つけ': 0.3333333333333333,
  'エアコン': 0.3333333333333333,
  '方': 0.3333333333333333},
 {'エアコン': 0.25, '上げ': 0.25, '方': 0.25, '温度': 0.25},
 {'つけ': 0.3333333333333333,
  'テレビ': 0.3333333333333333,
  '方': 0.3333333333333333}]